# Curate DataFrames and AnnDatas

Curating datasets typically means three things:

1. Validate: ensure a dataset meets predefined _validation criteria_
2. Standardize: transform a dataset so that it meets validation criteria, e.g., by fixing typos or using standardized identifiers
3. Annotate: link a dataset against metadata records

In LaminDB, valid metadata is metadata that's stored in a metadata registry and _validation criteria_ merely defines a mapping onto a field of a registry.

```{admonition} Example

`"Experiment 1"` is a valid value for `ULabel.name` if a record with this name exists in the {class}`~lamindb.ULabel` registry.

```

In [ ]:
# !pip install 'lamindb[bionty]'
!lamin init --storage ./test-curate --schema bionty

## Validate a DataFrame

Let's start with a DataFrame that we'd like to validate.

In [ ]:
import lamindb as ln
import bionty as bt
import pandas as pd


df = pd.DataFrame(
    {
        "temperature": [37.2, 36.3, 38.2],
        "cell_type": ["cerebral pyramidal neuron", "astrocytic glia", "oligodendrocyte"],
        "assay_ontology_id": ["EFO:0008913", "EFO:0008913", "EFO:0008913"],
        "donor": ["D0001", "D0002", "D0003"]
    },
    index = ["obs1", "obs2", "obs3"]
)
df

Define validation criteria and create a {class}`~lamindb.Curator` object.

In [ ]:
# in the dictionary, each key is a column name of the dataframe, and each value
# is a registry field onto which values are mapped
categoricals = {
    "cell_type": bt.CellType.name,
    "assay_ontology_id": bt.ExperimentalFactor.ontology_id,
    "donor": ln.ULabel.name,
}

# pass validation criteria
curate = ln.Curator.from_df(df, categoricals=categoricals)

The {meth}`~lamindb.core.BaseCurator.validate` method checks our data against the defined criteria. It identifies which values are already validated (exist in our registries) and which are potentially problematic (do not yet exist in our registries).

In [ ]:
curate.validate()

In [ ]:
# check the non-validated terms
curate.non_validated

For `cell_type`, we saw that "cerebral pyramidal neuron", "astrocytic glia" are not validated.

First, let's standardize synonym "astrocytic glia" as suggested

In [ ]:
curate.standardize("cell_type")

In [ ]:
# now we have only one non-validated term left
curate.non_validated

For "cerebral pyramidal neuron", let's understand which cell type in the public ontology might be the actual match.

In [ ]:
# to check the correct spelling of categories, pass `public=True` to get a lookup object from public ontologies
# use `lookup = curate.lookup()` to get a lookup object of existing records in your instance
lookup = curate.lookup(public=True)
lookup

In [ ]:
# here is an example for the "cell_type" column
cell_types = lookup["cell_type"]
cell_types.cerebral_cortex_pyramidal_neuron

In [ ]:
# fix the cell type
df.cell_type = df.cell_type.replace({"cerebral pyramidal neuron": cell_types.cerebral_cortex_pyramidal_neuron.name})

For donor, we want to add the new donors: "D0001", "D0002", "D0003"

In [ ]:
# this adds donors that were _not_ validated
curate.add_new_from("donor")

In [ ]:
# validate again
validated = curate.validate()
validated

## Validate an AnnData

Here we additionally specify which `var_index` to validate against.

In [ ]:
import anndata as ad

X = pd.DataFrame(
    {
        "ENSG00000081059": [1, 2, 3], 
        "ENSG00000276977": [4, 5, 6], 
        "ENSG00000198851": [7, 8, 9], 
        "ENSG00000010610": [10, 11, 12], 
        "ENSG00000153563": [13, 14, 15],
        "ENSGcorrupted": [16, 17, 18]
    }, 
    index=df.index
)

adata = ad.AnnData(X=X, obs=df)
adata

In [ ]:
curate = ln.Curator.from_anndata(
    adata, 
    var_index=bt.Gene.ensembl_gene_id,  # validate var.index against Gene.ensembl_gene_id
    categoricals=categoricals, 
    organism="human",
)

In [ ]:
curate.validate()

Non-validated terms can be accessed via:

In [ ]:
curate.non_validated

Subset the `AnnData` to validated genes only:

In [ ]:
adata_validated = adata[:, ~adata.var.index.isin(curate.non_validated["var_index"])].copy()

Now let's validate the subsetted object:

In [ ]:
curate = ln.Curator.from_anndata(
    adata_validated, 
    var_index=bt.Gene.ensembl_gene_id,  # validate var.index against Gene.ensembl_gene_id
    categoricals=categoricals, 
    organism="human",
)

curate.validate()

## Save a curated artifact

The validated object can be subsequently saved as an {class}`~lamindb.Artifact`:

In [ ]:
artifact = curate.save_artifact(description="test AnnData")

Saved artifact has been annotated with validated features and labels:

In [ ]:
artifact.describe()

We've walked through the process of validating, standardizing, and annotating datasets going through these key steps:

1. Defining validation criteria
2. Validating data against existing registries
3. Adding new validated entries to registries
4. Annotating artifacts with validated metadata

By following these steps, you can ensure your data is standardized and well-curated.

If you have datasets that aren't DataFrame-like or AnnData-like, read: {doc}`curate-any`.